<h1> <strong> ORDERLINES CLEANING </strong> <h1>

<h2> <strong> Initial exploration  </strong>  </h2>

In [1]:
import pandas as pd

orderlines = pd.read_csv("orderlines.csv")
print(orderlines)

             id  id_order  product_id  product_quantity      sku unit_price  \
0       1119109    299539           0                 1  OTT0133      18.99   
1       1119110    299540           0                 1  LGE0043     399.00   
2       1119111    299541           0                 1  PAR0071     474.05   
3       1119112    299542           0                 1  WDT0315      68.39   
4       1119113    299543           0                 1  JBL0104      23.74   
...         ...       ...         ...               ...      ...        ...   
293978  1650199    527398           0                 1  JBL0122      42.99   
293979  1650200    527399           0                 1  PAC0653     141.58   
293980  1650201    527400           0                 2  APP0698       9.99   
293981  1650202    527388           0                 1  BEZ0204      19.99   
293982  1650203    527401           0                 1  APP0927      13.99   

                       date  
0       2017-01-01 00

In [2]:
ol = orderlines

In [8]:
ol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


In [9]:
ol.describe()

,id,id_order,product_id,product_quantity
count,2.939830e+05,293983.000000,293983.0,293983.000000
mean,1.397918e+06,419999.116544,0.0,1.121126
std,1.530096e+05,66344.486479,0.0,3.396569
min,1.119109e+06,241319.000000,0.0,1.000000
25%,1.262542e+06,362258.500000,0.0,1.000000
50%,1.406940e+06,425956.000000,0.0,1.000000
75%,1.531322e+06,478657.000000,0.0,1.000000
max,1.650203e+06,527401.000000,0.0,999.000000


<h2> <strong> Data set  </strong>  </h2>

In [13]:
ol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                293983 non-null  int64 
 1   id_order          293983 non-null  int64 
 2   product_id        293983 non-null  int64 
 3   product_quantity  293983 non-null  int64 
 4   sku               293983 non-null  object
 5   unit_price        293983 non-null  object
 6   date              293983 non-null  object
dtypes: int64(4), object(3)
memory usage: 15.7+ MB


<p> Analyzing the data seems like  there are two columns to fix: price and date </p>

In [18]:
ol['date'] = pd.to_datetime(ol['date'])

In [20]:
ol['unit_price'] = ol['unit_price'].str.rsplit('.',1).apply(lambda x : x[0]+ ',' + x[1]).str.replace('.','',regex=True).str.replace(',','.',regex=True)

In [21]:
ol['unit_price'] = ol['unit_price'].astype(float)

In [22]:
ol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  float64       
 6   date              293983 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 15.7+ MB


<h2> <strong> Missing values  </strong>  </h2>

In [10]:
ol.isna().sum()

id                  0
id_order            0
product_id          0
product_quantity    0
sku                 0
unit_price          0
date                0
dtype: int64

<h2> <strong> Duplicates  </strong>  </h2>

In [11]:
ol.shape

(293983, 7)

In [12]:
ol.duplicated().sum()

0

<h> <strong> We can save it as a new csv <strong> </h>

In [24]:
ol.to_csv('orderlines_cl.csv', index=False)

<h2> <strong> Duplicates  </strong>  </h2>

<h>  We can identify some outliers based on deeper analysis between orders and orderlines  </h>

In [26]:
import pandas as pd

orders = pd.read_csv("orders.csv")

In [27]:
o = orders

In [25]:
ol['total_price'] = ol['product_quantity'] * ol['unit_price']

In [28]:
o_info = (
ol
    .groupby('id_order')
    .agg({'total_price':'sum'})
    .merge(o, how='left', left_on='id_order', right_on='order_id')
    
)
o_info

,total_price,order_id,created_date,total_paid,state
0,44.99,241319.0,2017-01-02 13:35:40,44.99,Cancelled
1,135.98,NaN,NaN,NaN,NaN
2,129.16,241423.0,2017-11-06 13:10:02,136.15,Completed
3,10.77,242832.0,2017-12-31 17:40:03,15.76,Completed
4,77.99,243330.0,2017-02-16 10:59:38,84.98,Completed
...,...,...,...,...,...
204850,42.99,527397.0,2018-03-14 13:56:38,42.99,Place Order
204851,42.99,527398.0,2018-03-14 13:57:25,42.99,Shopping Basket
204852,141.58,527399.0,2018-03-14 13:57:34,141.58,Shopping Basket
204853,19.98,527400.0,2018-03-14 13:57:41,19.98,Shopping Basket


In [31]:
o_info = o_info.dropna()
o_info

,total_price,order_id,created_date,total_paid,state
0,44.99,241319.0,2017-01-02 13:35:40,44.99,Cancelled
2,129.16,241423.0,2017-11-06 13:10:02,136.15,Completed
3,10.77,242832.0,2017-12-31 17:40:03,15.76,Completed
4,77.99,243330.0,2017-02-16 10:59:38,84.98,Completed
6,153.87,243784.0,2017-11-24 13:35:19,157.86,Cancelled
...,...,...,...,...,...
204850,42.99,527397.0,2018-03-14 13:56:38,42.99,Place Order
204851,42.99,527398.0,2018-03-14 13:57:25,42.99,Shopping Basket
204852,141.58,527399.0,2018-03-14 13:57:34,141.58,Shopping Basket
204853,19.98,527400.0,2018-03-14 13:57:41,19.98,Shopping Basket


In [32]:
o_info['price_difference'] = o_info['total_paid'] - o_info['total_price']
o_info

<ipython-input-32-ff5b10167578>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  o_info['price_difference'] = o_info['total_paid'] - o_info['total_price']


,total_price,order_id,created_date,total_paid,state,price_difference
0,44.99,241319.0,2017-01-02 13:35:40,44.99,Cancelled,0.00
2,129.16,241423.0,2017-11-06 13:10:02,136.15,Completed,6.99
3,10.77,242832.0,2017-12-31 17:40:03,15.76,Completed,4.99
4,77.99,243330.0,2017-02-16 10:59:38,84.98,Completed,6.99
6,153.87,243784.0,2017-11-24 13:35:19,157.86,Cancelled,3.99
...,...,...,...,...,...,...
204850,42.99,527397.0,2018-03-14 13:56:38,42.99,Place Order,0.00
204851,42.99,527398.0,2018-03-14 13:57:25,42.99,Shopping Basket,0.00
204852,141.58,527399.0,2018-03-14 13:57:34,141.58,Shopping Basket,0.00
204853,19.98,527400.0,2018-03-14 13:57:41,19.98,Shopping Basket,0.00


In [42]:
o_info= o_info.query('-0.1<=price_difference<=200').sort_values('price_difference')
o_info

,total_price,order_id,created_date,total_paid,state,price_difference
11258,1511.73,311619.0,2017-01-18 14:01:04,1511.64,Shopping Basket,-0.09
95994,15030.19,404898.0,2017-09-26 15:25:00,15030.10,Shopping Basket,-0.09
121626,3080.00,433907.0,2017-11-23 10:18:12,3079.93,Shopping Basket,-0.07
56699,18780.00,360920.0,2017-05-31 13:39:20,18779.93,Shopping Basket,-0.07
51067,3110.49,354777.0,2017-05-11 10:18:33,3110.43,Shopping Basket,-0.06
...,...,...,...,...,...,...
193017,158.00,514134.0,2018-02-19 11:29:35,320.98,Completed,162.98
439,74.99,299658.0,2017-01-01 12:32:16,239.05,Shopping Basket,164.06
191363,54.00,512215.0,2018-02-15 19:51:44,222.95,Completed,168.95
253,52.99,296036.0,2017-01-23 19:23:53,233.97,Completed,180.98


In [43]:
o_info.to_csv('orderlines_no_outliers.csv', index=False)